In [1]:
import pandas as pd
import os 
from TSMixerAPI import TSMixer

2025-02-06 21:38:56.704003: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
lengths = [90,420]
num_series = [50,300]
strctures = ['stationary','trend']
int_type = 'hom'
data_type = 'sim'
seasonality_period = 30
pred_len = 24
data_root_path = base_dir = os.path.join(os.getcwd(),'../../../')

In [3]:
model = TSMixer()
model.compile(learning_rate=1e-4, loss='mae')
for iter in range(model.args.itr):
    for len in lengths:
        for series in num_series:
            for strcture in strctures:
                data_name = data_type + '_' + str(len) + '_' + str(series) + '_' + strcture + '_' + int_type
                print(f"Training {TSMixer.__name__} on {data_name} with pred_len {pred_len}")

                model.fit(
                iter = iter + 1,
                data_type = data_type,
                data_name=data_name,
                data_root_path=data_root_path,
                batch_size=32,
                epochs=100,
                seasonality_period = seasonality_period,
                pred_len=pred_len,
                features = 'M',
                )

                predictions = model.predict()

                print(f'\nMoving to next...\n')

Training TSMixer on sim_90_50_stationary_hom with pred_len 24
Beginning to fit the model with the following arguments:
{'seed': 100, 'features': 'M', 'target': 'TARGET', 'checkpoints': './checkpoints', 'delete_checkpoint': False, 'seq_len': 12, 'n_block': 2, 'ff_dim': 2048, 'dropout': 0.05, 'norm_type': 'B', 'activation': 'relu', 'temporal_dim': 16, 'hidden_dim': 64, 'num_workers': 0, 'itr': 3, 'loss': 'mae', 'learning_rate': 0.0001, 'patience': 3, 'data_name': 'sim_90_50_stationary_hom', 'data_type': 'sim', 'root_path': '/home/RDC/shaharor/Schreibtisch/causality_analysis/src/models/tsmixer/../../../', 'seasonality_period': 30, 'treatment_rate': (None,), 'pred_len': 24, 'batch_size': 32, 'train_epochs': 100, 'iter': 1}


2025-02-06 21:40:05.317831: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2025-02-06 21:40:05.630253: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA A40 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 84 deviceMemorySize: 44.45GiB deviceMemoryBandwidth: 648.29GiB/s
2025-02-06 21:40:05.631555: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA A40 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 84 deviceMemorySize: 44.45GiB deviceMemoryBandwidth: 648.29GiB/s
2025-02-06 21:40:05.632773: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:ca:00.0 name: NVIDIA A40 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 84 deviceMemorySize: 44.45GiB deviceMemoryBandwidth: 648.29GiB/s
2025-02-06 21:40:05.634006: I t

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.3516 - mae: 1.3516 - val_loss: 1.3689 - val_mae: 1.3689

Epoch 00001: val_loss improved from inf to 1.36889, saving model to ./checkpoints/TSMixer_sim_sim_90_50_stationary_hom_M_sl12_pl24_iter1_best
Epoch 2/100
1/1 [==============================] - 0s 95ms/step - loss: 1.3421 - mae: 1.3421 - val_loss: 1.3490 - val_mae: 1.3490

Epoch 00002: val_loss improved from 1.36889 to 1.34899, saving model to ./checkpoints/TSMixer_sim_sim_90_50_stationary_hom_M_sl12_pl24_iter1_best
Epoch 3/100
1/1 [==============================] - 0s 96ms/step - loss: 1.3243 - mae: 1.3243 - val_loss: 1.3303 - val_mae: 1.3303

Epoch 00003: val_loss improved from 1.34899 to 1.33027, saving model to ./checkpoints/TSMixer_sim_sim_90_50_stationary_hom_M_sl12_pl24_iter1_best
Epoch 4/100
1/1 [==============================] - 0s 95ms/step - loss: 1.3076 - mae: 1.3076 - val_loss: 1.3127 - val_mae: 1.3127

Epoch 00004: val_loss improved from 1.33027 